In [ ]:
# !pip install python-dotenv
# !pip install transformers
# !pip install bitsandbytes
# !pip install accelerate
# !pip install sentence_transformers

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [2]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad,
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [3]:
from pprint import pprint

from torch import cuda, bfloat16

from transformers import pipeline
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [4]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/512M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
chatbot = pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

Demo

In [22]:
import pandas as pd

qa_data = pd.read_csv('/content/火影忍者.txt', header=None)
qa_data = qa_data[0].str.replace('\r\n', '\n').to_list()

In [23]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )

        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [24]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>水龍彈的結印順序是什麼
user: 水龍彈的結印順序是什麼
AI: 水龍彈的結印順序為：丑-申-卯-子-亥-酉-丑-午-酉-子-寅-戌-寅-巳-丑-未-巳-亥-未-子-申-酉-辰-酉-丑-午-未-寅-巳-子-申-卯-亥-辰-未-子-丑-申-酉-子-亥-酉。
>>>螺旋手裡劍很厲害嗎?主要是誰的招式?
user: 螺旋手裡劍很厲害嗎?主要是誰的招式?
AI: 風遁·螺旋丸是漩渦鳴人的招式。這個招式是鳴人在開發風遁·螺旋手裡劍的過程中衍生出來的術。螺旋丸的特點是利用風遁的力量，將劍給風化的概念，讓劍在空中形成螺旋狀，具有高精度和高速的攻擊能力。這個招式非常厲害，具有很高的攻擊力和靈活性，可以說是漩渦鳴人的招牌技能之一。
>>>我今天發現我是火屬性的查克拉，你會建議我學習哪個最厲害的忍術?
user: 我今天發現我是火屬性的查克拉，你會建議我學習哪個最厲害的忍術?
AI: 如果你是火屬性的查克拉，在風�的忍術家族中，風遁·螺旋丸是一個非常強大的選擇。由於火屬性對風�具有劣勢，學習風遁可以彌補這個劣勢，並發揮出近中距離的強大攻擊力。此外，風遁·螺旋丸的特性是利用風遁的力量，可以有效地應用在忍具或術上，具有很高的靈活性。因此，學習風遁·螺旋丸可以為火屬性的你帶來優勢。
>>>我想學習火遁，有推薦哪個最厲害的嗎?
user: 我想學習火遁，有推薦哪個最厲害的嗎?
AI: 如果你是火屬性的查克拉，在火�的忍術家族中，推薦學習火遁·火球。火球是火遁家族中具有代表性的忍術，它可以將所接觸到的東西全部燃燒殆盡，具有如火焰一般的高熱效果。此外，火球具有助燃的特性，在面對風屬性的對手時可以取得優勢。需要注意的是，火球會被水屬性的忍術所熄滅，所以在搭配其他忍術上要特別注意。
>>>我今天使出水遁．水流鞭，我該用什麼忍術做搭配?
user: 我今天使出水遁．水流鞭，我該用什麼忍術做搭配?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


AI: 如果你使出了水遁·水流鞭，可以考慮搭配使用水遁·雷鞭迅。水遁·雷鞭迅是跟水龍鞭搭配的雷遁忍術。這個忍術可以有效地克服水龍鞭的劣勢，提高水流鞭的攻擊力。水龍鞭可以集中水屬性的攻擊，而雷魁迅可以增加雷屬性的攻擊，使其攻擊力更加強大。此外，搭配使用冰雨也能發揮較好的效果。
>>>bye
